# 1.0 Set Up Environment

## Imports

In [1]:
import google.generativeai as genai
import os
import PyPDF2
from dotenv import load_dotenv
import tkinter as tk
from tkinter import messagebox
import pandas as pd

import csv
from datetime import datetime

In [2]:
# Load API key from .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure generative AI model
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

## 1.1. Define Functions

### 1.1.1 Function to extract PDF and text files

In [3]:
def get_ref_pdf(ref_doc):
  with open(ref_doc, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    ref_text = ""
    for page in reader.pages:
      ref_text += page.extract_text()
  return ref_text

def get_ref_txt(ref_doc):
    with open(ref_doc, "r") as file:
      return file.read()

### 1.1.2 Functions to pass queries to LLM
ask jeeves:
send prompt to LLM

security check:
Select security features

Gate check:
Search DUT for compliance of security features
Recommend the implementation if not present.
Save this assessment

pack suitcase:
Iterate through all of the signals and create the test cases categories requested by the user.
Save the test cases

In [ ]:
def ask_jeeves(prompt):
  response = model.generate_content(prompt)
  return response.text


def gate_check(des_doc):
  prompt_gate = "Your response must be bounded from [DOC1]\n\
  List all input and output signals of the design separated by commas \n"
  query_gate = prompt_gate + "[DOC1]: \n" + des_doc
  sig_names = ask_jeeves(query_gate)
  return sig_names.split(",")

def relationships(des_doc):
  prompt_gate = "Your response must be bounded from [DOC1] and any recollection of dependency graphs from computer architecture. \n\
  Create a table to show the depedency between signals.  \n"
  query_gate = prompt_gate + "[DOC1] =design specifications: \n" + des_doc
  response = ask_jeeves(query_gate)
  print(response)

def pack_suitcase(des_ref, sig_dep, sig_name):  
  date = str(datetime.now().date())
  file_title = f"HTAX_Outputs/{sig_name}_Test_Cases_{date}.txt"
  
  prompt_test_case = f"Your response must be bounded from [DOC1], [DOC2] and knowledge of defining test cases for Hardware Verificaion. \
  Define test cases for the {sig_name} signal to demonstrate timing and signal dependencies of the signal. \
  The response must be a table that includes: Test case ID, test case description, stimulus description, and description of the expected behavior. \
  [DOC2] = signal depency: {sig_dep} \n\
  [DOC1] = design specificaitons: {des_ref}"
  test_cases  = ask_jeeves(prompt_test_case)
  with open(file_title, "a") as file:
    file.write(str(test_cases))
  print(test_cases)

def assert_asiento(test_case_name, test_case, des_ref, assert_if):
  prompt_sva = (
  f"Your response must be bounded from [DOC1], [DOC2], "
  f"and prior knowledge of SystemVerilog Assertions for Hardware Verification. "
  f"Generate additional assertions for [DOC2] to check the expected behavior for all {test_case_name} test cases using the signals already delcared in [DOC2]"
  f"Only generate the assertion, do not recreate the file"
  f"Each assertion must map back to a test case"
  f"{test_case_name}: {test_case} \n"
  f"[DOC1] = Design Specification\n{des_ref}\n"
  f"[DOC2] = interface file\n{assert_if}\n"
  )
  date = str(datetime.now().date())
  file_title = f"HTAX_Outputs/{test_case_name}_Assertions_{date}.txt"
  gen_assert=(ask_jeeves(prompt_sva))
  with open(file_title, "a") as file:
    file.write(str(gen_assert))
  print(gen_assert)

  def security_check(selected_items, des_ref, sec_ref):
    # Generate timestamp
    date = str(datetime.now())
    file_title = f"AES/Security Assessment{date}.txt"
    prompt_sec = f"\nYour response must be bounded from [DOC0] and [DOC1] \n\
    Does the design defined in [DOC1] have any of the following features defined in [DOC0]:{selected_items}? \n\
    The response should be in table format to map the design feature to the security feature"

    query =  "[DOC0]= FPGA Security \n" + sec_ref + "\n[DOC1]=AES Design Specification \n" + des_ref + prompt_sec
    response = f'{selected_items}' + '\n' + ask_jeeves(query)

    # Write to file
    with open(file_title, "w") as file:
        file.write(response)
    print(response)

### 1.1.3 Create Window to pass user defined inputs to LLM

In [5]:
def select_sec_feat(std = None):
    selected_items = []
    _std = std

    def close():
        root.quit()
        #root.destroy()

    def get_sec_feat(std):
        if std is None or 'nsa':
            sec_feat = pd.read_csv("/Users/mchatter/Documents/DEng/ITDE/Python/Project/Reference Documents/NSA Features2.csv")
        elif std == 'cwe':
            sec_feat = pd.read_csv("/Users/mchatter/Documents/DEng/ITDE/Python/Project/Reference Documents/NSA Features2.csv")
        #print(sec_feat.head())
        else:
            print('incorrect request')
        sec_feat = sec_feat.iloc[:,1].unique()
        return sec_feat

    def generate_list():
        #selected_items = []
        nonlocal selected_items
        for item, var in checkbutton_vars.items():
            if var.get() == 1:
                selected_items.append(item)
        root.quit() 
                
    root = tk.Tk()
    root.title("FPGA Security Guidance")

    label = tk.Label(root, text="Select the applicable secuirty features to check for in the FPGA-based design. ", bg="black")
    label.pack(pady=5)

    items = get_sec_feat(_std)
    checkbutton_vars = {}

    for item in items:
        var = tk.IntVar()
        checkbutton = tk.Checkbutton(root, text=item, variable=var)
        checkbutton.pack()
        checkbutton_vars[item] = var

    generate_button = tk.Button(root, text="Check for Features", command=generate_list)
    generate_button.pack()

    quit_button = tk.Button(root, text="Quit", command=close)
    quit_button.pack()

    root.mainloop()

    print("Selected items are: ", selected_items)

    return selected_items

In [6]:
def create_input_window(user_prompt):
    def save_and_close():
        nonlocal saved_input
        saved_input = text_box.get("1.0", tk.END).strip()
        if not saved_input:
            messagebox.showwarning("Warning", "Input cannot be empty!")
        else:
            root.destroy()

    # Create the main window
    root = tk.Tk()
    root.title("Input Window")
    root.geometry("400x200")
    root.configure(bg="lightgray")  # Change window background color

    label = tk.Label(root, text=user_prompt, bg="black")
    label.pack(pady=5)

    # Create the Textbox with a white background
    text_box = tk.Text(root, height=5, width=30, fg="black", bg="white")
    text_box.pack(pady=10)

    # Create the Save and Close button
    done_button = tk.Button(root, text="Done", command=save_and_close)
    done_button.pack(pady=5)

    # Run the application
    saved_input = ""
    root.mainloop()
    
    return saved_input


## 1.2. Load Files

In [7]:
des_if = "HTAX/htax_tx_interface.txt"
htax_pdf = "HTAX/Spec.pdf"

### 1.2.1 Convert files to readable text.

In [8]:
assert_if = get_ref_txt(des_if)
#uncomment design
des_ref = get_ref_pdf(htax_pdf)

#des_ref = get_ref_txt(ip_md % ip_block[2])

# 2.0 Security Checkpoint

Check for NSA specific security features in design

In [ ]:
selected_items = select_sec_feat()

Selected items are:  ['Encyption Algorithms', 'Device Secret Key Storage', 'Key Rolling', 'Side Channel Attack Protection', 'Secure Boot']


In [ ]:
selected_items = ['Tamper Detection','Secure Memory','Configuration File Encryption']
selected_items = ['Key Rolling, Side Channel Attack Protection, Secure Boot, Encryption Algorithms, Device Secret Key']

In [ ]:
security_check(selected_items, des_ref, nsa_ref)

['Encyption Algorithms', 'Device Secret Key Storage', 'Key Rolling', 'Side Channel Attack Protection', 'Secure Boot']
This document provides detailed specifications and design considerations for an implementation of a masked AES (Advanced Encryption Standard) cipher core. The mask is applied to protect against side-channel attacks, including power analysis and timing attacks.

**Masking Scheme**

The masking scheme employed in this implementation is 1st-order masking, which means that both the cipher and key expand data paths use two shares. This provides protection against first-order side-channel attacks, such as power analysis and timing attacks.

**Data Path Architecture**

The AES unit uses a parallel data path to increase noise in the system and make it more resistant to higher-order side-channel attacks. The 128-bit parallel data path uses up to 20 S-Boxes (16 inside SubBytes, 4 inside KeyExpand) that are evaluated in parallel.

**Control Path Protection**

To protect against fa

In [ ]:
security_check(selected_items, top_ref, nsa_ref)

['Encyption Algorithms', 'Device Secret Key Storage', 'Key Rolling', 'Side Channel Attack Protection', 'Secure Boot']
This appears to be a technical document related to the Earlgrey chip, specifically its pin configuration and RTL implementation. Here's a summary of the main points:

**Pin Configuration**

* The document describes the pin configuration of the Earlgrey chip, including the type of each pad (e.g., BidirStd, BidirOd) and their connections.
* There are open drain and virtual open drain pads, which support high-impedance states and protection circuits.

**RTL Implementation Notes**

* The top-level netlist for Earlgrey is a combination of hand-written SystemVerilog RTL with auto-generated sections.
* A script generates the interconnecting crossbar (via TLUL) and instantiations at the top level.
* The document notes that there is subtlety in the notion of hierarchy within the top-level module.

**FPGA Platform**

* In the FPGA platform, the logic for JTAG and SPI device are m

# 3.0 Gate Check

Extract signal names. Store for debugging

In [10]:
sig_names = gate_check(des_ref)

In [11]:
with open("HTAX_Outputs/SigNames.csv", "a", newline='') as file:
    writer = csv.writer(file)
    writer.writerow(sig_names)

In [12]:
print(sig_names)

['tx_outport_req[PORTS-1:0]', ' tx_vc_req[VC-1:0]', ' tx_data[W-1:0]', ' tx_sot[VC-1:0]', ' tx_eot', ' tx_release_gnt', ' tx_vc_gnt[VC-1:0]', ' rx_vc_req[VC-1:0]', ' rx_vc_gnt[VC-1:0]', ' rx_data[WIDTH-1:0]', ' rx_sot[VC-1:0]', ' rx_eot\n']


# 4.0. Departure

## 4.1 Suitcases and Test Cases

Define test case categories to check for based on the design spec.

In [13]:
relationships(des_ref)

Based solely on the provided [DOC1], a table showing signal dependencies in the HTAX is difficult to create comprehensively.  The document describes signals and their behavior but doesn't explicitly define a complete dependency graph in a tabular format. However, we can illustrate some key dependencies based on the text:

| Signal              | Depends On                                                                     | Affects                                                                          | Notes                                                                                                                              |
|----------------------|---------------------------------------------------------------------------------|----------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------|
| `tx_outport_req`    | N/A (I

In [ ]:
sig_name = "tx_eot"
sig_dep = f"Signal: {sig_name} | Depends on: | Description: SEnd of transaction signal. "

test_case = pack_suitcase(des_ref, sig_dep, sig_name)
print(test_case)

Based on the provided documentation ([DOC1], [DOC2]),  defining comprehensive test cases for `tx_eot` requires more detail on the HTAX's operational characteristics and the surrounding environment (e.g.,  the behavior of `tx_release_gnt`, the interaction with other signals like `tx_data`, and the handling of various data packet sizes and transaction lengths). However, we can outline some fundamental test cases focusing on timing and signal dependencies:


| Test Case ID | Test Case Description | Stimulus Description | Expected Behavior |
|---|---|---|---|
| TC_TX_EOT_001 | Verify `tx_eot` assertion with a single data packet transaction. |  Assert `tx_vc_req` and `tx_outport_req` for a single virtual channel. After receiving `tx_vc_gnt`, assert `tx_sot` and a single data packet on `tx_data`.  | `tx_eot` is asserted for one clock cycle simultaneously with the last data packet.  `tx_release_gnt` is asserted one clock cycle before `tx_eot`. |
| TC_TX_EOT_002 | Verify `tx_eot` assertion wit

In [ ]:
test_case_name = "TC_TX_EOT"
#test_case = 
'''
| Test Case ID | Test Case Description | Stimulus Description | Expected Behavior |
|---|---|---|---|
| TC_TX_EOT_01 | Verify `tx_eot` assertion with a single data packet transaction. |  A single data packet is transmitted.  `tx_vc_req`, `tx_outport_req`, and `tx_sot` are asserted for a single virtual channel.  `tx_data` carries the single packet. | `tx_eot` is asserted for one clock cycle simultaneously with the last data packet (`tx_data`).  `tx_release_gnt` is asserted one clock cycle before `tx_eot`. |
| TC_TX_EOT_02 | Verify `tx_eot` assertion with a multi-packet transaction. | Multiple data packets are transmitted on a single virtual channel. `tx_vc_req`, `tx_outport_req`, and `tx_sot` are asserted. `tx_data` carries multiple packets.  | `tx_eot` is asserted for one clock cycle simultaneously with the last data packet (`tx_data`). `tx_release_gnt` is asserted one clock cycle before `tx_eot`. |
| TC_TX_EOT_03 | Verify `tx_eot` assertion with back-to-back transactions. | Two consecutive transactions are transmitted. The second transaction's `tx_vc_req` and `tx_outport_req` are asserted concurrently with the first transaction's `tx_release_gnt`. | `tx_eot` is asserted for one clock cycle for each transaction simultaneously with the last data packet of each transaction.  `tx_release_gnt` is asserted one clock cycle before each `tx_eot`.|
| TC_TX_EOT_04 | Verify `tx_eot` assertion with Minimal Latency Feature (MLF). | A single data packet is transmitted using the MLF.  `tx_sot` is asserted concurrently with `tx_vc_req` and `tx_outport_req`. Only one virtual channel is requested. | `tx_eot` is asserted for one clock cycle simultaneously with the last data packet (`tx_data`). `tx_release_gnt` is asserted one clock cycle before `tx_eot`.  |
| TC_TX_EOT_05 | Verify `tx_eot` assertion timing. | Measure the delay between the assertion of the last data packet and the assertion of `tx_eot`. | The delay should be minimal, ideally one clock cycle (as per the specification). |
| TC_TX_EOT_06 | Verify `tx_eot` deassertion.  | A transaction is completed. | `tx_eot` should deassert after one clock cycle. |
| TC_TX_EOT_07 | Verify `tx_eot` and `tx_release_gnt` relationship. |  A transaction is completed. | `tx_release_gnt` is asserted one clock cycle before `tx_eot`. |

'''

In [16]:
assert_asiento(test_case_name, test_case, des_ref, assert_if)

```systemverilog
   // -----------------------------------------
   // TC_TX_EOT_01: Single packet transaction
   // -----------------------------------------
   property single_packet_eot;
      @(posedge clk) disable iff(!rst_n)
      (tx_data) ##1 tx_eot;
   endproperty
   assert_tc_tx_eot_01 : assert property(single_packet_eot)
   else $error("TC_TX_EOT_01 failed: tx_eot not asserted simultaneously with last data packet.");


   // ------------------------------------------
   // TC_TX_EOT_02: Multi-packet transaction
   // ------------------------------------------
   property multi_packet_eot;
      @(posedge clk) disable iff(!rst_n)
      (tx_data) [->1] ##1 tx_eot; // Matches last tx_data in a sequence
   endproperty
   assert_tc_tx_eot_02 : assert property(multi_packet_eot)
   else $error("TC_TX_EOT_02 failed: tx_eot not asserted simultaneously with last data packet.");


   // ----------------------------------------------
   // TC_TX_EOT_03: Back-to-back transactions
   // -